In [1]:
import pylast
import pandas as pd
from pathlib import Path
import requests
from IPython.display import Image, display
import ast
import os
from glob import glob
from tqdm.auto import tqdm

In [9]:
tqdm.pandas()

/home/zach/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
API_KEY = "e22d3ef6eb85b8b554fc0a7e3832cd75"  # this is a sample key
API_SECRET = "7e575d3a57cb58e19ebbba4dbee919db"

# In order to perform a write operation you need to authenticate yourself
username = "zacharyneveu"
password_hash = "817ff0e30b8425e4bb511be5969a7115"

network = pylast.LastFMNetwork(api_key=API_KEY, api_secret=API_SECRET,
                               username=username, password_hash=password_hash)

In [3]:
datap = Path("/home/zach/Data/fma_medium/fma_meta")

In [4]:
df = pd.read_csv(datap/"raw_tracks.csv")

FileNotFoundError: [Errno 2] File b'/home/zach/Data/fma_medium/fma_meta/raw_tracks.csv' does not exist: b'/home/zach/Data/fma_medium/fma_meta/raw_tracks.csv'

In [5]:
df.head()

NameError: name 'df' is not defined

In [6]:
def fix_genres(genre_string):
    """ Turns strings in genre dataframe into a single-genre dictionary
    """
    try:
        return ast.literal_eval(genre_string)[0]
    except Exception as E:
        return None

In [23]:
df['genre'] = df.apply(lambda row: fix_genres(row.track_genres), axis=1)

In [34]:
print(glob("covers/Electronic/Taitsu*"))

['covers/Electronic/Taitsu [Tights].png']


In [40]:
def sanitize_fn(fn):
    """ Sanitizes file names (remove [] and / so they work with globs)
    """
    for ch in ['[', ']']:
        if ch in fn:
            fn = fn.replace(ch, '')
    return fn

In [ ]:
for loc,row in tqdm(df.iterrows(), total=df.shape[0]):
    try:
        if row.genre is None:
            continue
        genre = row.genre['genre_title']
        artist, title = str(row.artist_name), str(row.album_title)
        os.makedirs("covers/"+genre, exist_ok=True)
        fn = sanitize_fn("covers/"+genre+"/"+title.replace('/', '_'))
        if len(glob(fn+"*")) > 0:
            # skip re-downloading covers
            continue
        album = network.get_album(artist, title)
        try:
            cover = album.get_cover_image()
            if cover is not None:
                img = requests.get(cover)
                fn += '.'+cover.split('.')[-1]
                with open(fn, 'wb') as fh:
                    fh.write(img.content)
                display(Image(filename=fn))
            else:
                Path(fn+".txt").touch()
                #print(f"Cover not found for {title} by {artist}")
        except Exception as E:
            if E is KeyboardInterrupt:
                break
            else:
                Path(fn+".txt").touch()
                #print(f"{title} not found")
    except Exception as E:
        if E is KeyboardInterrupt:
            break
        else:
            print(loc)
            print(E)

In [ ]:
!rm -r *.txt

In [41]:
len(glob("covers/Metal/Black Elk*"))

1